In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt
IMAGE_SIZE=224
BATCH_SIZE=32
CHANNELS=3
EPOCHS=10

In [ ]:
dataset_dir = '/content/drive/MyDrive/data'
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 4188 files belonging to 4 classes.


In [ ]:
class_names=dataset.class_names
class_names

['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [ ]:
len(dataset)

131

In [ ]:
train_size=0.8
len(dataset)*train_size

104.80000000000001

In [ ]:
train_ds=dataset.take(105)

In [ ]:
test_ds=dataset.skip(105)

In [ ]:
len(test_ds)

26

In [ ]:
val_size=0.1
len(dataset)*val_size

13.100000000000001

In [ ]:
test_ds=test_ds.skip(13)

In [ ]:
def get_ds_partitions(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=get_ds_partitions(dataset)

In [ ]:
len(train_ds)

104

In [ ]:
len(val_ds)

13

In [ ]:
len(test_ds)

14

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resiz=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])
dataug=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=4
model=models.Sequential([
    resiz,
    dataug,
    layers.Conv2D(96,11,strides=(4,4),padding='valid',input_shape=input_shape,activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    layers.BatchNormalization(),
    layers.Conv2D(256,11,strides=(1,1),padding='valid',activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    layers.BatchNormalization(),
    layers.Conv2D(384,3,strides=(1,1),padding='valid',activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(384,3,strides=(1,1),padding='valid',activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256,3,strides=(1,1),padding='valid',activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(units=4096,activation='relu'),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(units=4096,activation='relu'),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(units=1000,activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(units=38,activation='softmax')
])
model.build(input_shape=input_shape)

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (32, 62, 62, 96)          34944     
                                                                 
 max_pooling2d_3 (MaxPoolin  (32, 31, 31, 96)          0         
 g2D)                                                            
                                                                 
 batch_normalization_8 (Bat  (32, 31, 31, 96)          384       
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (32, 21, 21, 256)        

In [ ]:
model.compile(
    optimizer='adamax',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
EPOCHS=50

In [ ]:
history=model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/50
104/104 [==============================] - 12s 49ms/step - loss: 1.4779 - accuracy: 0.6610 - val_loss: 7.0964 - val_accuracy: 0.3966
Epoch 2/50
104/104 [==============================] - 5s 46ms/step - loss: 0.7735 - accuracy: 0.7548 - val_loss: 1.2218 - val_accuracy: 0.5625
Epoch 3/50
104/104 [==============================] - 5s 47ms/step - loss: 0.6136 - accuracy: 0.7702 - val_loss: 1.0807 - val_accuracy: 0.6250
Epoch 4/50
104/104 [==============================] - 5s 46ms/step - loss: 0.5831 - accuracy: 0.7858 - val_loss: 0.8388 - val_accuracy: 0.6947
Epoch 5/50
104/104 [==============================] - 5s 47ms/step - loss: 0.5376 - accuracy: 0.7951 - val_loss: 1.4196 - val_accuracy: 0.6731
Epoch 6/50
104/104 [==============================] - 5s 47ms/step - loss: 0.4860 - accuracy: 0.8150 - val_loss: 0.7057 - val_accuracy: 0.6803
Epoch 7/50
104/104 [==============================] - 5s 46ms/step - loss: 0.4462 - accuracy: 0.8180 - val_loss: 0.6118 - val_accuracy: 0.810

In [ ]:
scores=model.evaluate(test_ds)

14/14 [==============================] - 0s 17ms/step - loss: 0.2820 - accuracy: 0.9196


In [ ]:
scores

[0.282027930021286, 0.9196428656578064]

In [ ]:
from sklearn.metrics import classification_report
true_labels = []
predicted_labels = []

for batch in test_ds:
    images, labels = batch
    predictions = model.predict(images)
    predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
    true_labels.extend(labels.numpy())
report = classification_report(true_labels, predicted_labels, target_names=class_names)
print(report)

1/1 [==============================] - 0s 31ms/step
                precision    recall  f1-score   support

        Blight       0.83      0.93      0.88       117
   Common_Rust       0.95      0.97      0.96       152
Gray_Leaf_Spot       0.83      0.51      0.63        49
       Healthy       0.98      1.00      0.99       130

      accuracy                           0.92       448
     macro avg       0.90      0.85      0.87       448
  weighted avg       0.92      0.92      0.91       448



#for 10

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_mat = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(confusion_mat)


Confusion Matrix:
[[ 75   0  65   0]
 [  5 109  10   1]
 [ 41   0  24   0]
 [ 53   0  22  43]]
